# Run pyCIAM

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from warnings import catch_warnings, filterwarnings

from pyCIAM.run import execute_pyciam
from shared import (
    AUTHOR,
    CONTACT,
    DIR_SCRATCH,
    HISTORY,
    PATH_OUTPUTS,
    PATH_PARAMS,
    PATH_REFA,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    RES_VERS,
    start_dask_cluster,
)

In [3]:
TMPPATH = DIR_SCRATCH / f"pyCIAM_results_quantiles_prechunked_{RES_VERS}.zarr"

# optimized for the Code Ocean machines
SEG_CHUNKSIZE = 9

SEG_ADM_SUBSET = None

DESCRIPTION = "Projected coastal damages from pyCIAM, using quantiles of SLR scenarios."

In [4]:
client, cluster = start_dask_cluster()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 124.37 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36301,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 124.37 GiB
Comm: tcp://127.0.0.1:41707,Total threads: 1
Dashboard: http://127.0.0.1:39787/status,Memory: 7.77 GiB
Nanny: tcp://127.0.0.1:34293,


In [5]:
!rm -r {str(TMPPATH)}

In [ ]:
with catch_warnings():
    filterwarnings("ignore", module="distributed.worker.memory")
    execute_pyciam(
        PATH_PARAMS,
        PATH_SLIIDERS,
        [PATH_SLR_INT],
        ["is"],
        PATH_REFA,
        econ_input_path_seg=PATH_SLIIDERS_SEG,
        output_path=PATH_OUTPUTS if SEG_ADM_SUBSET is None else None,
        seg_var_subset=SEG_ADM_SUBSET,
        surge_input_paths=PATHS_SURGE_LOOKUP,
        tmp_output_path=TMPPATH,
        pyciam_seg_chunksize=SEG_CHUNKSIZE,
        quantiles=[0.5],
        dask_client_func=lambda: client,
        remove_tmpfile=False,
        extra_attrs={
            "author": AUTHOR,
            "contact": CONTACT,
            "description": DESCRIPTION,
            "history": HISTORY,
        },
    )